# Exercise 1: Familiarize yourself with `pandas`
If you are new to Python, check out the introductory Python courses available through Caltrans's LinkedIn Learning Library:
* https://www.linkedin.com/learning/search?keywords=python&u=36029164

Skills: 
* `pandas` is one of the base Python packages for working with tabular data.
* Do some grouping and aggregation. Many ways to do this!
* Export to Google Cloud Storage
* Practice committing on GitHub

References: 
* https://docs.calitp.org/data-infra/analytics_new_analysts/01-data-analysis-intro.html
* https://docs.calitp.org/data-infra/analytics_tools/saving_code.html

In [4]:
import pandas as pd
import altair as alt

## Read in an Excel file from GCS
* F-String
* How to open a workbook with multiple sheets

In [3]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/starter_kit/"

In [ ]:
FILE = "starter_kit_csis_scoring_workbook.xlsx"

In [ ]:
# Combine to get the proper file name 

In [ ]:
sheet_names = []

In [ ]:
df = pd.read_excel()

## Merge

## Groupby / Aggregation

* By Caltrans District, calculate the average CSIS score and find difference between max and min score.
* Hint: for `pandas`: `groupby / agg`, `pivot_table`, `groupby / transform`.

## Export to Google Cloud Storage (GCS)

* Make sure credential works
* Use this path: "gs://calitp-analytics-data/data-analyses/FILENAME"
* Export using `df.to_parquet()` and `df.to_csv()`

## Make a Chart 
* Read in the parquet file from GCS using F Strings.
* Make a visualization using one `Altair`
    * Requirements: use a color palette from Cal-ITP
    * The chart must be in Century-Gothic font.
    * Size it to be 600 pixels wide, 400 pixels high

## You're almost done!
* Name this notebook `YOURNAME_exercise1.ipynb`
    * If you need to rename because you already named it, do it within the terminal.
    * `git mv OLDNAME.ipynb NEWNAME.ipynb`. 
    * The `mv` stands for move, and renaming a file is basically "moving" its path. Doing it this way retains the git history associated with the notebook. If you rename directly with right click, rename, you destroy the git history.
* Use a descriptive commit message (ex: adding chart, etc). GitHub already tracks who makes the commit, the date, the timestamp of it, the files being affected, so your commit message should be more descriptive than the metadata already stored.